In [86]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
import pickle

In [87]:
data = pd.read_csv("Churn_Modelling.csv")
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [88]:
# data = data.drop(data.columns[0:3], axis = 1, inplace = True )
# data

data.drop(data.columns[0:3], axis=1, inplace=True)



In [89]:
le = LabelEncoder()
data['Gender'] = le.fit_transform(data['Gender'])
data

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...
9995,771,France,1,39,5,0.00,2,1,0,96270.64,0
9996,516,France,1,35,10,57369.61,1,1,1,101699.77,0
9997,709,France,0,36,7,0.00,1,0,1,42085.58,1
9998,772,Germany,1,42,3,75075.31,2,1,0,92888.52,1


In [90]:
oe = OneHotEncoder(handle_unknown= 'ignore', sparse_output= False)
geo = oe.fit_transform(data[['Geography']])
geo_df = pd.DataFrame(geo, columns= oe.get_feature_names_out(['Geography']))
geo_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


In [91]:
data = pd.concat([data.drop('Geography', axis = 1), geo_df], axis = 1)
data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [92]:
# X = data.drop('EstimatedSalary', 'Exited', axis = 1)
# y = data['EstimatedSalary']

X = data.drop(['EstimatedSalary', 'Exited'], axis=1)
y = data['EstimatedSalary']


X_train,X_test, y_train, y_test = train_test_split(X, y, test_size= 0.2, random_state= 1)

In [93]:
X.columns


Index(['CreditScore', 'Gender', 'Age', 'Tenure', 'Balance', 'NumOfProducts',
       'HasCrCard', 'IsActiveMember', 'Geography_France', 'Geography_Germany',
       'Geography_Spain'],
      dtype='object')

In [94]:
print(X.shape)
print(y.shape)



(10000, 11)
(10000,)


In [95]:
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [96]:
with open('le_sal.pkl', 'wb') as file:
    pickle.dump(le, file)

with open('oe_sal.pkl', 'wb') as file:
    pickle.dump(oe, file)

with open('sc_sal.pkl', 'wb') as file:
    pickle.dump(sc, file)

In [97]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

model = Sequential([
    Dense(64, activation='relu', input_shape = (X_train.shape[1],)),
    Dense(32, activation='relu'),
    Dense(1, activation = 'linear')
])

model.compile(optimizer = 'adam', loss= 'mean_absolute_error', metrics = ['mae'])
model.summary()

d:\Projects_Aiml\Ann_Project\venv\Lib\site-packages\keras\src\layers\core\dense.py:106: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_12 (Dense)                │ (None, 64)             │           768 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_13 (Dense)                │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_14 (Dense)                │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,881 (11.25 KB)

 Trainable params: 2,881 (11.25 KB)

 Non-trainable params: 0 (0.00 B)

In [98]:
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
import datetime

log_dir = 'regression/fit/' + datetime.datetime.now().strftime('%Y%m%d-%H%M%S')
tensorboard_cb = TensorBoard(log_dir = log_dir, histogram_freq = 1)

In [99]:
early_cb = EarlyStopping(monitor='val_loss', patience= 10, restore_best_weights= True)

In [100]:
history = model.fit(
    X_train, y_train,
    validation_data = (X_test,y_test),
    epochs = 100,
    callbacks = [early_cb,tensorboard_cb]
)

Epoch 1/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 6s 9ms/step - loss: 100089.2266 - mae: 100089.2266 - val_loss: 99664.7188 - val_mae: 99664.7188
Epoch 2/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - loss: 99331.2578 - mae: 99331.2578 - val_loss: 98149.1172 - val_mae: 98149.1172
Epoch 3/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 96711.8438 - mae: 96711.8438 - val_loss: 94352.5312 - val_mae: 94352.5312
Epoch 4/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 91575.1250 - mae: 91575.1250 - val_loss: 87967.0469 - val_mae: 87967.0469
Epoch 5/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 84031.0469 - mae: 84031.0469 - val_loss: 79609.1406 - val_mae: 79609.1406
Epoch 6/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 75053.7188 - mae: 75053.7188 - val_loss: 70548.4453 - val_mae: 70548.4453
Epoch 7/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 66211.9453 - mae: 66211.9453 - val_loss: 62532.3047 - val_mae: 62532.3047
Epoch 8/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step 

In [101]:
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [102]:
#twice
%tensorboard --logdir regression/fit/

Reusing TensorBoard on port 6006 (pid 596), started 1:12:42 ago. (Use '!kill 596' to kill it.)

In [103]:
test_loss, test_mae = model.evaluate(X_test, y_test)
print(f'Test MAE: {test_mae}')

63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 49725.7031 - mae: 49725.7031
Test MAE: 49725.703125


In [104]:
model.save('regression_model.h5')